<a href="https://colab.research.google.com/github/alinnman/lineofsight/blob/main/mallorca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple sample for line of sight
This sample calculates the line of sight from a target (lighthouse/mountain/skyscraper) to one observer located at a specific elevation above the sea surface. All **heights** and **distances** are specified in meters. Calculations are made without and with refraction. For refraction **air pressure** (kPa), **temperature** (degrees Celsius) and **temperature gradient** (degrees Celsius / meter) must be entered. Default values are pre-set.

A calculation for an observer at the same position, but at sea level is also made, and a check is made to see if the parabola approximation ("8 inches per miles squared") is correct or not.

A calculation of obstructed height and horizon dip is also made.

Instructions for use: Fill in the parameters and **Press Ctrl-F9 or the arrow button below** to calculate the result which will be presented in the Results cell below.

*The pre-set values are taken from the view from Grandfather's Mountain towards Charlotte (North Carolina, USA)*.

Allow some time for the first execution (Google Colab needs some time to start up).<br/>
When running for the first time you may get a warning about the code not originating from Google Colab (it is stored in GitHub). You can safely ignore this warning, but you may of course read the source code, to check it is safe. Click "Show code" to inspect the source code (you may also modify).

*Tip: To simulate severe refraction set the temperature gradient to 0.1 (i.e strong temperature inversion)*.

The used maths (formulas etc.) is presented in the bottom cell.

In [ ]:
# @markdown ### Enter parameters:
TARGET_HEIGHT = 1436 # @param {"type":"integer"}
TARGET_DISTANCE = 272000 # @param {"type":"integer"}
OBSERVER_HEIGHT = 1813 # @param {"type":"integer"}
PRESSURE = 101 # @param {"type":"integer"}
TEMPERATURE = 20 # @param {"type":"integer"}
TEMP_GRADIENT = -0.01 # @param {"type":"number"}

In [ ]:
# @title
from math import sqrt, pi, atan2, cos, acos
from IPython.display import Markdown, display

EARTH_CIRCUMFERENCE_EQUATORIAL = 40075.017
EARTH_CIRCUMFERENCE_MERIDIONAL = 40007.86
EARTH_CIRCUMFERENCE = (EARTH_CIRCUMFERENCE_EQUATORIAL + EARTH_CIRCUMFERENCE_MERIDIONAL) / 2
EARTH_RADIUS = EARTH_CIRCUMFERENCE / (2 * pi)

R = EARTH_RADIUS*1000

def visibility_limit (RR : float, h1 : float, h2 : float,\
                      get_obscurations : bool = False,
                      target_distance = None)\
    -> tuple [float, float, list[tuple[float,float]]]:
  ''' Geometry for line-of-sight '''
  x1a = sqrt (((RR + h1)**2) - RR**2)
  x1r = atan2 (x1a, RR)
  x1 = x1r * RR
  x2a = sqrt (((RR + h2)**2) - RR**2)
  x2r = atan2 (x2a, RR)
  x2 = x2r * RR

  obscurations = list()
  obscuredHeight = None
  output_target_done = False

  # Do split and get obscured height
  if get_obscurations and target_distance is not None:
    for i in range (10 + 1):
      s = x1r * ((10 - i) / 10)
      ho = RR / cos (s) - RR
      dist = x2 + s*RR
      if dist < target_distance and not output_target_done:
        dist2 = target_distance - x2
        angle = dist2 / RR
        hoPrim = RR / cos (angle) - RR
        obscurations.append ((target_distance, hoPrim, True))
        obscuredHeight = hoPrim
        output_target_done = True
      obscurations.append ((dist, ho, False))
    if not output_target_done:
      obscurations.append ((target_distance, 0.0, True))

  return x1, x2, obscurations

def reportObsc (obsc : list[tuple[float,float, bool]]) -> float:
  display (Markdown ("### Obscured heights (with refraction)"))
  tableString = "| Distance (m) | Obstructed Height (m) | Visible part (%) |\n"
  tableString += "| ----- | ----- | ----- |\n"

  visiblePercentage = None
  zeroReached = False

  for e in obsc:
    distance = e[0]
    distanceString = str(round(distance))
    obscHeight = e[1]
    obscTarget = e[2]
    if obscHeight == 0 and distance > 0 and not zeroReached:
      distanceString = distanceString + " and closer"
      zeroReached = True
    fattyMarker = ""
    if obscTarget:
      fattyMarker = "**"
    if obscHeight < 0:
      percent = 0.0
      obscHeight = float(TARGET_HEIGHT)
    else:
      percent = 100.0 - (obscHeight / TARGET_HEIGHT) * 100.0
    if percent < 0:
      percent = 0.0
    elif percent > 100:
      percent = 100.0
    if obscTarget:
      visiblePercentage = percent
    tableString += "|" + fattyMarker + distanceString + fattyMarker +\
                   "|" + fattyMarker + str(round(obscHeight,1)) + fattyMarker + \
                   "|" + fattyMarker + str(round(percent,1)) + fattyMarker + "|\n"
  display (Markdown(tableString))
  return visiblePercentage

def get_dip_of_horizon (hm : int | float, RR : float)\
      -> float:
    the_dip = acos (RR/(RR+hm))*(180/pi)*60
    return the_dip

DT_DH = TEMP_GRADIENT # Temperature shift with increasing elevation
K_FACTOR = 503*(PRESSURE*10)*(1/((TEMPERATURE+273)**2))*(0.0343 + DT_DH)
Ra = R / (1 - K_FACTOR) # This is the radius adjusted for refraction
# See https://en.wikipedia.org/wiki/Atmospheric_refraction#cite_note-Hirt2010-28

h1 = TARGET_HEIGHT # Height of target

display (Markdown ("# Results"))
display (Markdown ("### HEIGHT of TARGET = " + str(h1) + " meters"))
display (Markdown ("### TARGET DISTANCE = " + str(TARGET_DISTANCE) + " meters"))

for h2 in [0, OBSERVER_HEIGHT]: # Try some different observer elevations

  display (Markdown ("### CALCULATING for OBSERVER HEIGHT = " + str(h2) + " meters."))

  ## No refraction

  x1, x2, obsc = visibility_limit (R, h1, h2)
  distance = x1 + x2
  dipOfHorizon = get_dip_of_horizon (h2, R)
  lightHouseTable = "|Refraction|Line of Sight (meters)|Dip of Horizon (arcminutes)\n"
  lightHouseTable += "|-----|-----|-----|\n"
  lightHouseTable += "|No|" + str(round(distance))+ "|" +str(round(dipOfHorizon))+ "\n"

  # For zero elevation (observer at sea level) we can double-check with the parabola approximation
  # NOTE: The parabola approximation is useless if observer is above sea level!
  if h2 == 0:
    outputStr = "*Checking the 8 inches per miles squared approximation*<br/>"
    MILES_PER_KM = 0.621371
    miles = (distance/1000) * MILES_PER_KM
    inches = 8 * (miles**2)
    INCHES_PER_METER = 39.3701
    meters_from_inches = inches / INCHES_PER_METER
    outputStr += "*Parabola approximation of target height = " + str(round(meters_from_inches,5)) + " meters, from a distance of " +str(round(distance))+" meters.*<br/>"
    outputStr += "*.. which is " + \
                 str(round(abs(round(((meters_from_inches - h1) / h1),6)*100),6)) + \
                 " % off the real value*"
    display (Markdown (outputStr))

  ## Now adjust for refraction

  x1,x2,obsc = visibility_limit (Ra, h1, h2, get_obscurations=True, target_distance=TARGET_DISTANCE)
  distance = x1 + x2
  dipOfHorizon = get_dip_of_horizon (h2, Ra)

  lightHouseTable += "|Yes|" + str(round(distance))+ "|" +str(round(dipOfHorizon))+ "|\n"
  display (Markdown(lightHouseTable))
  if obsc is not None:
    vp = reportObsc (obsc)
    if vp == 0:
      vpString = "#### <span style=\"color:red\">**Target is completely obscured**</span>"
    elif vp < 100.0:
      vpString = "#### <span style=\"color:orange\">**Target is partially obscured (" + str(round(vp,1)) + "% visible)**</span>"
    else:
      vpString = "#### <span style=\"color:green\">**Target is fully visible**</span>"
    display (Markdown(vpString))
  display (Markdown("<hr>"))


# Results

### HEIGHT of TARGET = 1436 meters

### CALCULATING for OBSERVER HEIGHT = 0 meters

*Checking the 8 inches per miles squared approximation*<br/>*Parabola approximation of target height = 1435.68157 meters, from a distance of 135275 meters.*<br/>*.. which is 0.0222 % off the real value*

|Refraction|Line of Sight (meters)|Dip of Horizon (arcminutes)
|-----|-----|-----|
|No|135275|0
|Yes|146196|0|


...

|Refraction|Line of Sight (meters)|Dip of Horizon (arcminutes)
|-----|-----|-----|
|No|287269|82
|Yes|310462|76|


### Obscured heights (with refraction)

| Distance (m) | Obstructed Height (m) | Visible part (%) |
| ----- | ----- | ----- |
|310462|1436.0|0.0|
|295842|1163.1|19.0|
|281222|919.0|36.0|
|**272000**|**779.8**|**45.7**|
|266603|703.6|51.0|
|251983|516.9|64.0|
|237364|359.0|75.0|
|222744|229.7|84.0|
|208125|129.2|91.0|
|193505|57.4|96.0|
|178885|14.4|99.0|
|164266 and closer|0.0|100.0|


#### <span style="color:orange">**Target is partially obscured (45.7% visible)**</span>

<hr>

<br/><br/><br/><br/>

## Math formulas

<br/>

#### The **line of sight** $d_L$ is calculated using this formula

$$d_L = R_r \times \left( \arctan\frac{\sqrt{(R_r+h_1)^2 - {R_r}^2}}{R_r} + \arctan\frac{\sqrt{(R_r+h_2)^2 - {R_r}^2}}{R_r} \right)$$

Where:

$R_r$ is Earth radius. (It may be a value corrected for refraction)<br/>
$h_1$ is Target elevation<br/>
$h_2$ is Observer elevation

<br/>

#### The **obscured height** $h_o$ is calculated using this formula

$$h_o = \frac{R_r}{\cos{\left(\frac{d_t - \left( \arctan{\frac{\sqrt{{\left(R_r + h_2\right)}^2-R_r^2}}{R_r}} \times R_r \right)}{R_r}\right)}} - R_r$$

Where:

$d_t$ is the **target distance**<br/>
$R_r$ is Earth radius. (It may be a value corrected for refraction)<br/>
$h_2$ is Observer elevation

<br/>

#### **Refraction** is calculated using an enlarged Earth radius $R_a$ according to this formula

$$R_a = \frac{R}{1 - 503 \times P \times \frac{1}{{\left( T+273 \right) }^2} \times \left( 0.0343 + \frac{dT}{dH}\right)}$$

Where:

$R$ is Earth radius. <br/>
$P$ is Air Pressure (in millibars)<br/>
$T$ is Temperature (in degrees Celsius)<br/>
$\frac{dT}{dH}$ is Temperature Gradient (degrees celsius per meter). Normal conditions is about 1 degree colder for each 100 m, i.e. $-0.01$.

<br/>

#### **Dip of the Horizon**

$$a_{\text{dip}} = \arccos{\frac{R_r}{R_r + h_2}}$$

Where:

$R_r$ is Earth radius. (It may be a value corrected for refraction)<br/>
$h_2$ is Observer elevation
